In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import pickle as pkl
import notebook_helper
import sys
import numpy as np
import imodelsx
import imodelsx.process_results
import sasc.data.data
from sasc.data.data import TASKS_D3
import sasc.methods.m4_evaluate as m4_evaluate
import warnings
import scipy.stats
def sem(x):
    '''Compute standard error of the mean, ignoring NaNs
    '''
    with warnings.catch_warnings():
        return scipy.stats.sem(x, ddof=0)
tqdm.pandas()
TASK_NAMES = list(TASKS_D3.keys())
sys.path.append('../experiments/')
# results_dir = '/home/chansingh/mntv1/mprompt/feb18_synthetic_sweep'
results_dir = '/home/chansingh/mntv1/mprompt/feb25'

In [ ]:
r = pd.read_pickle('../results/r.pkl')

In [ ]:
# simplifying
default_params = {
    'noise_ngram_scores': 0,
    'module_num_restrict': -1,
}
rd = r
for k, v in default_params.items():
    rd = rd[rd[k] == v]

# Top results

In [ ]:
with pd.option_context('display.max_colwidth', -1, 'display.max_rows', 200):
    d = (
        rd[[
            'task_keyword', 'top_explanation_init_strs',
            'top_contains_keywords', #        'mean_contains_keywords',
            'top_score_synthetic', 'module_num', 'seed'
        ]]
        .rename(columns={
            'task_keyword': 'GT Explanation',
            'top_explanation_init_strs': 'Generated Explanation',
        })
    )
    d_ = d[d.top_contains_keywords == 0]
    slice_ = pd.IndexSlice[d_.index, d_.columns]

    # s = d.style
    display(
        d
        .style.hide()
        .set_properties(**{'background-color': '#132639'}, subset=slice_)
        .format(precision=2)
   )

# Details

**Selected examples**

In [ ]:
pos_examples = [(25, 2), (4, 1), (21, 1), (42, 1), (7, 1), (50, 1), (41, 1), (8, 1), (45, 1), (47, 1)]
neg_examples = [(18, 1), (26, 1), (39, 1), (2, 1)]

with pd.option_context('display.max_colwidth', -1, 'display.max_rows', 200):
    tab = (
        pd.concat([rd[(rd.module_num == x[0]) & (rd.seed == x[1])] for x in pos_examples + neg_examples])
        [[
                'task_keyword', 'top_explanation_init_strs',
            ]]
            .rename(columns={
                'task_keyword': 'Groundtruth Explanation',
                'top_explanation_init_strs': 'Generated Explanation',
            })
    )
    display(tab)
dummy = [''] * tab.shape[0]
dummy[0] = '\\parbox[c]{1mm}{\\multirow{' + str(len(pos_examples)) + '}{*}{\\rotatebox[origin=c]{90} {Correct}}}'
dummy[len(pos_examples)] = '\\parbox[c]{1mm}{\\multirow{' + str(len(neg_examples)) + '}{*}{\\rotatebox[origin=c]{90} {Incorrect}}}'
tab.insert(0, ' ', dummy)
print(tab.style.hide(axis="index").to_latex(hrules=True).replace('science \\\\', 'science \\\\ \n\midrule'))

**More details**

In [ ]:
# display full strings
with pd.option_context('display.max_colwidth', -1, 'display.max_rows', 200):
    display(
        rd
        .filter(['task_keyword', 'top_explanation_init_strs', 'top_contains_keywords', 'top_score_synthetic', 'module_num'])
        # .filter(['top_score_contains_keywords', 'top_score_synthetic', 'top_explanation_init_strs', 'explanation_init_ngrams', 'module_num'])
        # .filter(['top_explanation_init_strs', 'explanation_init_ngrams', 'top_strs_added'])
        # .filter(['explanation_init_strs', 'explanation_init_ngrams',])
        # .head(n=20)
    )